Importing needed packages for the project

requests and BeautifulSoup -> for sending HTTP requests and parsing the respond

pandas and numpy -> for data engineering and cleaning

pickle -> for saving the data locally from the RAM

os -> for handling local paths

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pickle as pkl
import os

Building Mining function which takes a list of URLs and Map variable to return data in it.
It parses data based on Aqarmap html files only

In [4]:
def data_mining(links, houses_data):
    '''
    links -> a list of URLs to send a HTTP request and parse the HTML response of it
    houses_data -> Empty map where data is returned and item for each link with the area as the Key and the list of found properties are the value
    '''
    for link in links:
        if(link['area'] not in houses_data.keys()):
            houses_data[link['area']] = []
        i=1
        while(len(houses_data[link['area']])<link['number_of_appartments']):
            url = f"{link['link']}?page={i}"
            request = requests.get(url)
            soup = BeautifulSoup(request.content, 'html.parser')
            data = soup.find_all('div', class_="search-listing-card")
            for h in data:
                id = h['id']
                title = h.find('h2')
                title = title.text.strip() if title is not None else ''
                price = h.find('div', class_="search-listing-card__price").find('span')
                price = price.text.strip() if price is not None else 0
                location = h.find('p', class_="search-listing-card__address").find('span')
                location = location.text if location is not None else ''
                attributes = h.find('div', class_="search-listing-card__attributes").find_all('label')
                attrib = []
                for a in attributes:
                    attrib.append(a.text.strip())
                size = attrib[0].split(' ') if len(attrib) > 0 else 0
                bedrooms = attrib[1] if len(attrib) > 1 else 1
                bathrooms = attrib[2] if len(attrib) > 2 else 1
                houses_data[link['area']].append({
                                    'id':id,
                                    'title':title,
                                    'price':price,
                                    'location':location,
                                    'size':size[0],
                                    'size_unit':size[1],
                                    'bedrooms':bedrooms,
                                    'bathrooms':bathrooms,
                                    })
            i+=1

Getting list of URLs of Alexandria properties based on Locations

In [ ]:
search_location = 'cairo/'
rent_of_sale = 'for-rent/'
'''
rent -> for-rent/,
sale -> for-sale/
'''
type_of_property = 'apartment/'
'''
apartment -> apartment/,
villa -> villa/,
furnished-apartment -> furnished-apartment/,
all -> property-type/
'''
website = 'https://aqarmap.com.eg/ar/'
request = requests.get(website+rent_of_sale+type_of_property+search_location)
soup = BeautifulSoup(request.content, 'html.parser')
data = soup.find('div', class_='sectionLoadMore').find_all('li')

Alexandria_appertmints_links = []
for l in data:
    loca = l.find('a')
    title = loca.text.split('(')
    Alexandria_appertmints_links.append({
                                        'area':title[0].strip(),
                                        'number_of_appartments':int(title[1].split(')')[0]),
                                        'link':f"https://aqarmap.com.eg{loca['href']}",
                                    })

In [ ]:
for area in Alexandria_appertmints_links:
    if('area' in area.keys()):
        print(area['area'] + "\t" + str(area['number_of_appartments']))

In [ ]:
Alex_data = {}
data_mining(Alexandria_appertmints_links, Alex_data)

In [ ]:
Alex_data.keys()

Building Dataframes and saving them to separate csv files for each location area

In [3]:
for key, data in Alex_data.items():
    df = pd.DataFrame(data)
    df.to_csv(f'Alex_properties\\{key}.csv', index=False)

Reading all the csv files and building a unified file containing all the properties

In [26]:
df = pd.read_csv('Alex_properties//أبو تلات.csv')
df['location_region'] = 'أبو تلات'
test=True

In [27]:
path = os.path.join(os.curdir,'Alex_properties')
for f in os.listdir(path):
    if test:
        test=False
        continue
    else:
        df2 = pd.read_csv(os.path.join(path, f))
        df2['location_region'] = f.split('.')[0]
        df = pd.concat([df, df2], axis=0)

In [30]:
df.to_csv('All_Alex_Properties.csv')